## 01. 항공권 가격 예측

* 항공권 티겟 가격을 예측하시오.
    * 제공된 데이터 목록: filght_train.csv, filght_test.csv
    * 예측할 컬럼: price

* 학습용 데이터(train)를 이용해 티켓가격을 예측하는 모델을 만든 후 이를 평가용 데이터(test)에 적용해 얻은 예측 값을 다음과 같은 형식의 CSV 파일로 생성하시오.

제출 파일은 다음 1개의 컬럼을 포함해야 한다.
* pred: 예측값(가격)
* 제출 파일명: 'result.csv'
제출한 모델의 성능은 RMSE 평가지표에 따라 채점한다.

* 제출 csv 파일명 및 형태: result.csv
```python
pred
56000
7000
11000
```

In [2]:
import pandas as pd
import numpy as np
import sklearn

In [5]:
df_train = pd.read_csv("../bigdata_analyst_cert/part2/ch8/flight_train.csv")
df_test = pd.read_csv("../bigdata_analyst_cert/part2/ch8/flight_test.csv")

print(df_train.shape)
print(df_test.shape)

(10505, 11)
(4502, 10)


In [6]:
df_train.head()

,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left,price
0,Vistara,UK-776,Kolkata,Evening,one,Late_Night,Delhi,Economy,6.58,31,7056
1,Vistara,UK-852,Bangalore,Morning,zero,Morning,Mumbai,Business,1.92,37,20760
2,Indigo,6E-2348,Delhi,Evening,one,Late_Night,Bangalore,Economy,5.58,25,3671
3,Air_India,AI-763,Kolkata,Early_Morning,one,Evening,Chennai,Business,12.00,15,55983
4,Indigo,6E-752,Hyderabad,Early_Morning,one,Evening,Chennai,Economy,9.50,20,5220


In [7]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10505 entries, 0 to 10504
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   airline           10505 non-null  object 
 1   flight            10505 non-null  object 
 2   source_city       10505 non-null  object 
 3   departure_time    10505 non-null  object 
 4   stops             10505 non-null  object 
 5   arrival_time      10505 non-null  object 
 6   destination_city  10505 non-null  object 
 7   class             10505 non-null  object 
 8   duration          10505 non-null  float64
 9   days_left         10505 non-null  int64  
 10  price             10505 non-null  int64  
dtypes: float64(1), int64(2), object(8)
memory usage: 902.9+ KB


In [8]:
df_train.describe()

,duration,days_left,price
count,10505.000000,10505.000000,10505.000000
mean,12.225536,26.050547,20650.139838
std,7.182264,13.539947,22570.924117
min,0.830000,1.000000,1105.000000
25%,6.750000,15.000000,4755.000000
50%,11.250000,26.000000,7455.000000
75%,16.170000,38.000000,42457.000000
max,40.500000,49.000000,110936.000000


In [16]:
ob_index = (df_train.dtypes == 'object').index
ob_index

Index(['airline', 'flight', 'source_city', 'departure_time', 'stops',
       'arrival_time', 'destination_city', 'class', 'duration', 'days_left',
       'price'],
      dtype='object')

In [18]:
df_train.nunique()

airline                6
flight              1153
source_city            6
departure_time         6
stops                  3
arrival_time           6
destination_city       6
class                  2
duration             392
days_left             49
price               3116
dtype: int64

In [24]:
numeric = ['duration', 'days_left', 'price']
categorical = ['airline', 'flight', 'source_city', 'departure_time', 'stops', 'arrival_time', 'destination_city', 'class']

In [26]:
df_train_numeric = df_train[numeric]
df_train_numeric.corr()

,duration,days_left,price
duration,1.000000,-0.050043,0.210738
days_left,-0.050043,1.000000,-0.092827
price,0.210738,-0.092827,1.000000


In [29]:
for ctg in categorical:
    print(f'{ctg} 컬럼의 유니크 밸류')
    print(df_train[ctg].unique())
    print()
    print(f'{ctg} nunique')
    print(df_train[ctg].nunique())
    print()
    print(f'{ctg} 컬럼의 유니크 걔수')
    print(df_train[ctg].value_counts())
    print('-'*50)

airline 컬럼의 유니크 밸류
['Vistara' 'Indigo' 'Air_India' 'AirAsia' 'GO_FIRST' 'SpiceJet']

airline nunique
6

airline 컬럼의 유니크 걔수
airline
Vistara      4425
Air_India    2842
Indigo       1556
GO_FIRST      800
AirAsia       564
SpiceJet      318
Name: count, dtype: int64
--------------------------------------------------
flight 컬럼의 유니크 밸류
['UK-776' 'UK-852' '6E-2348' ... '6E-7076' 'AI-435' 'SG-5002']

flight nunique
1153

flight 컬럼의 유니크 걔수
flight
UK-706     108
UK-832      99
UK-828      98
UK-720      98
UK-822      97
          ... 
G8-2502      1
6E-995       1
G8-287       1
6E-7118      1
SG-438       1
Name: count, Length: 1153, dtype: int64
--------------------------------------------------
source_city 컬럼의 유니크 밸류
['Kolkata' 'Bangalore' 'Delhi' 'Hyderabad' 'Mumbai' 'Chennai']

source_city nunique
6

source_city 컬럼의 유니크 걔수
source_city
Mumbai       2112
Delhi        2074
Bangalore    1865
Kolkata      1623
Hyderabad    1418
Chennai      1413
Name: count, dtype: int64
---------------------

In [ ]:
df_train['flight'].value_counts(ascending=True).iloc[:50]

flight
G8-719     1
6E-319     1
G8-2601    1
G8-2502    1
6E-995     1
G8-287     1
6E-7118    1
SG-438     1
6E-448     1
6E-7583    1
6E-453     1
6E-6464    1
SG-5002    1
SG-6027    1
6E-2032    1
6E-546     1
6E-2097    1
I5-881     1
SG-8104    1
SG-8483    1
AI-688     1
AI-713     1
AI-531     1
AI-9609    1
AI-604     1
6E-2007    1
6E-6517    1
6E-234     1
UK-627     1
6E-5343    1
6E-5336    1
SG-5012    1
6E-7224    1
6E-575     1
G8-2440    1
6E-782     1
6E-146     1
SG-242     1
SG-480     1
6E-172     1
SG-401     1
6E-6798    1
I5-1622    1
6E-7062    1
6E-809     1
AI-643     1
6E-5316    1
6E-6234    1
6E-161     1
6E-356     1
Name: count, dtype: int64

In [36]:
# 쿨하게 flight 드랍하시죠? 

df_train = df_train.drop('flight', axis=1)
df_train.head()

,airline,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left,price
0,Vistara,Kolkata,Evening,one,Late_Night,Delhi,Economy,6.58,31,7056
1,Vistara,Bangalore,Morning,zero,Morning,Mumbai,Business,1.92,37,20760
2,Indigo,Delhi,Evening,one,Late_Night,Bangalore,Economy,5.58,25,3671
3,Air_India,Kolkata,Early_Morning,one,Evening,Chennai,Business,12.00,15,55983
4,Indigo,Hyderabad,Early_Morning,one,Evening,Chennai,Economy,9.50,20,5220


In [52]:
from sklearn import preprocessing, model_selection

In [ ]:
help(model_selection)

In [ ]:
help(sklearn)

In [ ]:
help(pd.get_dummies)

In [49]:
df_label = df_train['price']
df_train = df_train.drop('price', axis=1)

In [53]:
df_train.head()

,airline,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left
0,Vistara,Kolkata,Evening,one,Late_Night,Delhi,Economy,6.58,31
1,Vistara,Bangalore,Morning,zero,Morning,Mumbai,Business,1.92,37
2,Indigo,Delhi,Evening,one,Late_Night,Bangalore,Economy,5.58,25
3,Air_India,Kolkata,Early_Morning,one,Evening,Chennai,Business,12.00,15
4,Indigo,Hyderabad,Early_Morning,one,Evening,Chennai,Economy,9.50,20


In [60]:
categorical.pop(1)

'flight'

In [61]:
dummies = pd.get_dummies(df_train[categorical])
dummies

,airline_AirAsia,airline_Air_India,airline_GO_FIRST,airline_Indigo,airline_SpiceJet,airline_Vistara,source_city_Bangalore,source_city_Chennai,source_city_Delhi,source_city_Hyderabad,...,arrival_time_Morning,arrival_time_Night,destination_city_Bangalore,destination_city_Chennai,destination_city_Delhi,destination_city_Hyderabad,destination_city_Kolkata,destination_city_Mumbai,class_Business,class_Economy
0,False,False,False,False,False,True,False,False,False,False,...,False,False,False,False,True,False,False,False,False,True
1,False,False,False,False,False,True,True,False,False,False,...,True,False,False,False,False,False,False,True,True,False
2,False,False,False,True,False,False,False,False,True,False,...,False,False,True,False,False,False,False,False,False,True
3,False,True,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,True,False
4,False,False,False,True,False,False,False,False,False,True,...,False,False,False,True,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10500,False,False,False,False,False,True,True,False,False,False,...,True,False,False,True,False,False,False,False,False,True
10501,False,False,False,False,False,True,False,False,False,False,...,False,True,False,False,False,False,False,True,False,True
10502,True,False,False,False,False,False,False,True,False,False,...,False,False,False,False,True,False,False,False,False,True
10503,False,False,False,False,False,True,False,False,False,False,...,False,True,False,False,False,False,True,False,True,False


In [62]:
df_train_dummies = pd.concat([df_train, dummies], axis=1)
df_train_dummies

,airline,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left,airline_AirAsia,...,arrival_time_Morning,arrival_time_Night,destination_city_Bangalore,destination_city_Chennai,destination_city_Delhi,destination_city_Hyderabad,destination_city_Kolkata,destination_city_Mumbai,class_Business,class_Economy
0,Vistara,Kolkata,Evening,one,Late_Night,Delhi,Economy,6.58,31,False,...,False,False,False,False,True,False,False,False,False,True
1,Vistara,Bangalore,Morning,zero,Morning,Mumbai,Business,1.92,37,False,...,True,False,False,False,False,False,False,True,True,False
2,Indigo,Delhi,Evening,one,Late_Night,Bangalore,Economy,5.58,25,False,...,False,False,True,False,False,False,False,False,False,True
3,Air_India,Kolkata,Early_Morning,one,Evening,Chennai,Business,12.00,15,False,...,False,False,False,True,False,False,False,False,True,False
4,Indigo,Hyderabad,Early_Morning,one,Evening,Chennai,Economy,9.50,20,False,...,False,False,False,True,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10500,Vistara,Bangalore,Evening,one,Morning,Chennai,Economy,16.83,11,False,...,True,False,False,True,False,False,False,False,False,True
10501,Vistara,Kolkata,Night,zero,Night,Mumbai,Economy,2.92,20,False,...,False,True,False,False,False,False,False,True,False,True
10502,AirAsia,Chennai,Night,one,Afternoon,Delhi,Economy,16.08,27,True,...,False,False,False,False,True,False,False,False,False,True
10503,Vistara,Mumbai,Evening,one,Night,Kolkata,Business,28.67,2,False,...,False,True,False,False,False,False,True,False,True,False


In [63]:
df_train_dummies = df_train_dummies.drop(categorical, axis=1)
df_train_dummies

,duration,days_left,airline_AirAsia,airline_Air_India,airline_GO_FIRST,airline_Indigo,airline_SpiceJet,airline_Vistara,source_city_Bangalore,source_city_Chennai,...,arrival_time_Morning,arrival_time_Night,destination_city_Bangalore,destination_city_Chennai,destination_city_Delhi,destination_city_Hyderabad,destination_city_Kolkata,destination_city_Mumbai,class_Business,class_Economy
0,6.58,31,False,False,False,False,False,True,False,False,...,False,False,False,False,True,False,False,False,False,True
1,1.92,37,False,False,False,False,False,True,True,False,...,True,False,False,False,False,False,False,True,True,False
2,5.58,25,False,False,False,True,False,False,False,False,...,False,False,True,False,False,False,False,False,False,True
3,12.00,15,False,True,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,True,False
4,9.50,20,False,False,False,True,False,False,False,False,...,False,False,False,True,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10500,16.83,11,False,False,False,False,False,True,True,False,...,True,False,False,True,False,False,False,False,False,True
10501,2.92,20,False,False,False,False,False,True,False,False,...,False,True,False,False,False,False,False,True,False,True
10502,16.08,27,True,False,False,False,False,False,False,True,...,False,False,False,False,True,False,False,False,False,True
10503,28.67,2,False,False,False,False,False,True,False,False,...,False,True,False,False,False,False,True,False,True,False


In [69]:
from sklearn.preprocessing import StandardScaler

std = StandardScaler()

after_std = std.fit_transform(df_train_numeric)
after_std = pd.DataFrame(after_std, columns=df_train_numeric.columns)
after_std

,duration,days_left
0,-0.786076,0.365562
1,-1.434927,0.808716
2,-0.925314,-0.077592
3,-0.031403,-0.816183
4,-0.379500,-0.446888
...,...,...
10500,0.641119,-1.111619
10501,-1.295689,-0.446888
10502,0.536690,0.070126
10503,2.289702,-1.776350


In [68]:
df_train_numeric = df_train_numeric.drop('price', axis=1)

In [71]:
df_train_dummies[df_train_numeric.columns] = after_std
df_train_dummies

,duration,days_left,airline_AirAsia,airline_Air_India,airline_GO_FIRST,airline_Indigo,airline_SpiceJet,airline_Vistara,source_city_Bangalore,source_city_Chennai,...,arrival_time_Morning,arrival_time_Night,destination_city_Bangalore,destination_city_Chennai,destination_city_Delhi,destination_city_Hyderabad,destination_city_Kolkata,destination_city_Mumbai,class_Business,class_Economy
0,-0.786076,0.365562,False,False,False,False,False,True,False,False,...,False,False,False,False,True,False,False,False,False,True
1,-1.434927,0.808716,False,False,False,False,False,True,True,False,...,True,False,False,False,False,False,False,True,True,False
2,-0.925314,-0.077592,False,False,False,True,False,False,False,False,...,False,False,True,False,False,False,False,False,False,True
3,-0.031403,-0.816183,False,True,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,True,False
4,-0.379500,-0.446888,False,False,False,True,False,False,False,False,...,False,False,False,True,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10500,0.641119,-1.111619,False,False,False,False,False,True,True,False,...,True,False,False,True,False,False,False,False,False,True
10501,-1.295689,-0.446888,False,False,False,False,False,True,False,False,...,False,True,False,False,False,False,False,True,False,True
10502,0.536690,0.070126,True,False,False,False,False,False,False,True,...,False,False,False,False,True,False,False,False,False,True
10503,2.289702,-1.776350,False,False,False,False,False,True,False,False,...,False,True,False,False,False,False,True,False,True,False


In [72]:
df_label

0         7056
1        20760
2         3671
3        55983
4         5220
         ...  
10500     9241
10501     4499
10502     2477
10503    79148
10504     4756
Name: price, Length: 10505, dtype: int64

In [73]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df_train_dummies, df_label, test_size=0.3, random_state=42)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(7353, 37)
(3152, 37)
(7353,)
(3152,)


In [74]:
help(sklearn)

Help on package sklearn:

NAME
    sklearn - Configure global settings and get information about the working environment.

PACKAGE CONTENTS
    __check_build (package)
    _build_utils (package)
    _built_with_meson
    _config
    _cyutility
    _distributor_init
    _isotonic
    _loss (package)
    _min_dependencies
    base
    calibration
    cluster (package)
    compose (package)
    conftest
    covariance (package)
    cross_decomposition (package)
    datasets (package)
    decomposition (package)
    discriminant_analysis
    dummy
    ensemble (package)
    exceptions
    experimental (package)
    externals (package)
    feature_extraction (package)
    feature_selection (package)
    frozen (package)
    gaussian_process (package)
    impute (package)
    inspection (package)
    isotonic
    kernel_approximation
    kernel_ridge
    linear_model (package)
    manifold (package)
    metrics (package)
    mixture (package)
    model_selection (package)
    multiclass
    

In [75]:
from sklearn import model_selection
help(model_selection)

Help on package sklearn.model_selection in sklearn:

NAME
    sklearn.model_selection - Tools for model selection, such as cross validation and hyper-parameter tuning.

PACKAGE CONTENTS
    _classification_threshold
    _plot
    _search
    _search_successive_halving
    _split
    _validation
    tests (package)

CLASSES
    builtins.object
        sklearn.model_selection._search.ParameterGrid
        sklearn.model_selection._search.ParameterSampler
    sklearn.model_selection._classification_threshold.BaseThresholdClassifier(sklearn.base.ClassifierMixin, sklearn.base.MetaEstimatorMixin, sklearn.base.BaseEstimator)
        sklearn.model_selection._classification_threshold.FixedThresholdClassifier
        sklearn.model_selection._classification_threshold.TunedThresholdClassifierCV
    sklearn.model_selection._plot._BaseCurveDisplay(builtins.object)
        sklearn.model_selection._plot.LearningCurveDisplay
        sklearn.model_selection._plot.ValidationCurveDisplay
    sklearn.model_

In [79]:
from sklearn import metrics
help(metrics)

Help on package sklearn.metrics in sklearn:

NAME
    sklearn.metrics - Score functions, performance metrics, pairwise metrics and distance computations.

PACKAGE CONTENTS
    _base
    _classification
    _dist_metrics
    _pairwise_distances_reduction (package)
    _pairwise_fast
    _plot (package)
    _ranking
    _regression
    _scorer
    cluster (package)
    pairwise
    tests (package)

CLASSES
    builtins.object
        sklearn.metrics._dist_metrics.DistanceMetric
        sklearn.metrics._plot.confusion_matrix.ConfusionMatrixDisplay
        sklearn.metrics._plot.regression.PredictionErrorDisplay
    sklearn.utils._plotting._BinaryClassifierCurveDisplayMixin(builtins.object)
        sklearn.metrics._plot.det_curve.DetCurveDisplay
        sklearn.metrics._plot.precision_recall_curve.PrecisionRecallDisplay
        sklearn.metrics._plot.roc_curve.RocCurveDisplay
    
    class ConfusionMatrixDisplay(builtins.object)
     |  ConfusionMatrixDisplay(confusion_matrix, *, display_la

In [76]:
from sklearn import linear_model

help(linear_model)

Help on package sklearn.linear_model in sklearn:

NAME
    sklearn.linear_model - A variety of linear models.

PACKAGE CONTENTS
    _base
    _bayes
    _cd_fast
    _coordinate_descent
    _glm (package)
    _huber
    _least_angle
    _linear_loss
    _logistic
    _omp
    _passive_aggressive
    _perceptron
    _quantile
    _ransac
    _ridge
    _sag
    _sag_fast
    _sgd_fast
    _stochastic_gradient
    _theil_sen
    tests (package)

CLASSES
    sklearn.base.BaseEstimator(sklearn.utils._repr_html.base.ReprHTMLMixin, sklearn.utils._repr_html.base._HTMLDocumentationLinkMixin, sklearn.utils._metadata_requests._MetadataRequester)
        sklearn.linear_model._huber.HuberRegressor(sklearn.linear_model._base.LinearModel, sklearn.base.RegressorMixin, sklearn.base.BaseEstimator)
        sklearn.linear_model._logistic.LogisticRegression(sklearn.linear_model._base.LinearClassifierMixin, sklearn.linear_model._base.SparseCoefMixin, sklearn.base.BaseEstimator)
            sklearn.linear_m

In [80]:
from sklearn.linear_model import LinearRegression, ElasticNet
from sklearn.metrics import root_mean_squared_error

lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
lr_rmse = root_mean_squared_error(y_test, y_pred)
print(f'lr_rmse = {lr_rmse}')

elastic = ElasticNet()
elastic.fit(X_train, y_train)
y_pred = elastic.predict(X_test)
elastic_rmse = root_mean_squared_error(y_test, y_pred)
print(f'elastic net rmse = {elastic_rmse}')

lr_rmse = 6637.045965650744
elastic net rmse = 12928.914036912636


In [81]:
from sklearn import ensemble
help(ensemble)

Help on package sklearn.ensemble in sklearn:

NAME
    sklearn.ensemble - Ensemble-based methods for classification, regression and anomaly detection.

PACKAGE CONTENTS
    _bagging
    _base
    _forest
    _gb
    _gradient_boosting
    _hist_gradient_boosting (package)
    _iforest
    _stacking
    _voting
    _weight_boosting
    tests (package)

CLASSES
    sklearn.base.BaseEstimator(sklearn.utils._repr_html.base.ReprHTMLMixin, sklearn.utils._repr_html.base._HTMLDocumentationLinkMixin, sklearn.utils._metadata_requests._MetadataRequester)
        sklearn.ensemble._base.BaseEnsemble(sklearn.base.MetaEstimatorMixin, sklearn.base.BaseEstimator)
    sklearn.base.ClassifierMixin(builtins.object)
        sklearn.ensemble._bagging.BaggingClassifier(sklearn.base.ClassifierMixin, sklearn.ensemble._bagging.BaseBagging)
        sklearn.ensemble._gb.GradientBoostingClassifier(sklearn.base.ClassifierMixin, sklearn.ensemble._gb.BaseGradientBoosting)
        sklearn.ensemble._hist_gradient_boost

In [ ]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

rf = RandomForestRegressor()
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

In [84]:
def model_fit_metric(models, X_train, X_test, y_train, y_test):
    for model in models:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        rmse = root_mean_squared_error(y_test, y_pred)
        print(f'{model} 결과: {rmse}')
        print('-'*50)

In [85]:
from sklearn.linear_model import LinearRegression, ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import root_mean_squared_error

lr = LinearRegression()
elastic = ElasticNet()
rf = RandomForestRegressor()
gbr = GradientBoostingRegressor()

models = [lr, elastic, rf, gbr]

model_fit_metric(models, X_train, X_test, y_train, y_test)

LinearRegression() 결과: 6637.045965650744
--------------------------------------------------
ElasticNet() 결과: 12928.914036912636
--------------------------------------------------
RandomForestRegressor() 결과: 4269.156531295163
--------------------------------------------------
GradientBoostingRegressor() 결과: 4956.774380272118
--------------------------------------------------


In [86]:
df_test

,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left
0,Vistara,UK-778,Kolkata,Afternoon,one,Morning,Chennai,Business,18.58,35
1,Air_India,AI-764,Delhi,Evening,one,Late_Night,Bangalore,Economy,8.92,35
2,Air_India,AI-569,Chennai,Early_Morning,one,Evening,Bangalore,Economy,12.17,13
3,Vistara,UK-960,Mumbai,Morning,one,Evening,Kolkata,Economy,7.67,22
4,GO_FIRST,G8-302,Chennai,Early_Morning,one,Evening,Bangalore,Economy,14.67,22
...,...,...,...,...,...,...,...,...,...,...
4497,Air_India,AI-660,Mumbai,Evening,one,Morning,Chennai,Economy,14.92,6
4498,Vistara,UK-960,Mumbai,Morning,one,Night,Hyderabad,Economy,11.00,47
4499,Air_India,AI-442,Mumbai,Afternoon,one,Evening,Delhi,Business,3.75,26
4500,Vistara,UK-860,Hyderabad,Early_Morning,one,Evening,Mumbai,Economy,9.67,2


In [ ]:
df_test[['duration', 'days_left']] = std.transform(df_test[['duration', 'days_left']])

AttributeError: 'DataFrame' object has no attribute 'get_dummies'

In [ ]:
pd.get_dummies(df_test, prefix=)